In [159]:
import numpy as np
np.random.seed(222)

In [160]:
# Parameters
n = 5  # Grid size n x n
actions = ['up', 'right', 'down', 'left']  # Actions the agent can take
action_space = len(actions)

In [161]:
# Initialize Q-table: (agent_row, agent_col, package_row, package_col, carrying_flag, action) 
q_values = np.random.rand(n, n, n, n, 2, action_space)

In [162]:
# Rewards
delivery_reward = 80  
move_reward = -1  
pickup_reward = 20 

In [163]:
# Define the target location (B)
B = (n - 1, n - 1)

In [164]:
# Function to check if the state is terminal (i.e., package delivered)
def is_terminal_state(agent_row, agent_col, carrying):
    return (agent_row, agent_col) == B and carrying

In [165]:
# Function to choose a random, non-terminal starting location for the agent and package
def get_starting_locations():
    agent_row = np.random.randint(n)
    agent_col = np.random.randint(n)
    package_row = np.random.randint(n)
    package_col = np.random.randint(n)
    while (agent_row, agent_col) == B or (package_row, package_col) == B:
        agent_row = np.random.randint(n)
        package_row = np.random.randint(n)
        package_col = np.random.randint(n)
    return agent_row, agent_col, package_row, package_col

In [166]:
# Epsilon-greedy algorithm for choosing the next action
def get_next_action(agent_row, agent_col, package_row, package_col, carrying, epsilon):
    if np.random.rand() < epsilon:
        return np.argmax(q_values[agent_row, agent_col, package_row, package_col, carrying])
    else:
        return np.random.randint(action_space)

In [167]:
# Function to get the next location based on the chosen action
def get_next_location(agent_row, agent_col, action_index):
    new_row, new_col = agent_row, agent_col
    if actions[action_index] == 'up' and agent_row > 0:
        new_row -= 1
    elif actions[action_index] == 'right' and agent_col < n - 1:
        new_col += 1
    elif actions[action_index] == 'down' and agent_row < n - 1:
        new_row += 1
    elif actions[action_index] == 'left' and agent_col > 0:
        new_col -= 1
    return new_row, new_col

In [168]:
# Function to update the Q-values during training
#alpha - learning_rate, gamma - discount_factor
def update_q_values(old_state, action_index, reward, new_state, learning_rate, discount_factor, is_terminal_state):
    if not is_terminal_state:
        old_q_value = q_values[old_state][action_index]
        temporal_difference = reward + (discount_factor * np.max(q_values[new_state])) - old_q_value
        q_values[old_state][action_index] = old_q_value + (learning_rate * temporal_difference)
    else:
        old_q_value = q_values[old_state][action_index]
        temporal_difference = reward - old_q_value
        q_values[old_state][action_index] = old_q_value + (learning_rate * temporal_difference)

In [169]:
# Training parameters
epsilon = 0.9  # Epsilon for epsilon-greedy strategy
discount_factor = 0.9  # Discount factor for future rewards
learning_rate = 0.15  # Learning rate
num_episodes = 20000  # Number of training episodes
max_steps_per_episode = 200  # Limit the steps per episode

In [170]:
# Training loop
for episode in range(num_episodes):
    # Initialize starting locations
    agent_row, agent_col, package_row, package_col = get_starting_locations()
    carrying = 0  # Agent starts without carrying the package
    
    for step in range(max_steps_per_episode):
        # Choose action
        action_index = get_next_action(agent_row, agent_col, package_row, package_col, carrying, epsilon)
        
        # Get next location
        new_agent_row, new_agent_col = get_next_location(agent_row, agent_col, action_index)
        
        # Determine the reward and update carrying status
        if (new_agent_row, new_agent_col) == (package_row, package_col) and not carrying:
            reward = pickup_reward
            carrying = 1  # Now the agent is carrying the package
        elif (new_agent_row, new_agent_col) == B and carrying:
            reward = delivery_reward
        else:
            reward = move_reward
        
        # Update Q-values
        old_state = (agent_row, agent_col, package_row, package_col, carrying)
        new_state = (new_agent_row, new_agent_col, package_row, package_col, carrying)
        update_q_values(old_state, action_index, reward, new_state, learning_rate, discount_factor, is_terminal_state(agent_row, agent_col, carrying))
        
        # Transition to the new state
        agent_row, agent_col = new_agent_row, new_agent_col
        
        # Check if the task is complete
        if is_terminal_state(agent_row, agent_col, carrying):
            break
    
    

print('Training complete!')

Training complete!


In [171]:
# Test the agent's performance after training
def test_agent(num_tests=1000):
    success_count = 0
    for _ in range(num_tests):
        agent_row, agent_col, package_row, package_col = get_starting_locations()
        carrying = 0
        path = [(agent_row, agent_col)]
        for step in range(max_steps_per_episode):
            action_index = get_next_action(agent_row, agent_col, package_row, package_col, carrying, 1.0)
            agent_row, agent_col = get_next_location(agent_row, agent_col, action_index)
            path.append((agent_row, agent_col))
            if (agent_row, agent_col) == (package_row, package_col) and not carrying:
                carrying = 1
            if is_terminal_state(agent_row, agent_col, carrying):
                success_count += 1
                print('Success')
                break
        print(f'Path taken by agent for package location: {(package_row, package_col)} - ')
        print(path)
    print(f'Success rate: {success_count}/{num_tests}')

test_agent()

Success
Path taken by agent for package location: (4, 0) - 
[(1, 3), (2, 3), (3, 3), (4, 3), (4, 2), (4, 1), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]
Success
Path taken by agent for package location: (1, 4) - 
[(3, 3), (2, 3), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)]
Success
Path taken by agent for package location: (0, 0) - 
[(4, 3), (4, 2), (4, 1), (3, 1), (2, 1), (1, 1), (0, 1), (0, 0), (1, 0), (1, 1), (1, 2), (2, 2), (3, 2), (3, 3), (3, 4), (4, 4)]
Success
Path taken by agent for package location: (2, 4) - 
[(2, 1), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
Success
Path taken by agent for package location: (1, 4) - 
[(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)]
Success
Path taken by agent for package location: (0, 2) - 
[(3, 0), (2, 0), (2, 1), (1, 1), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (3, 4), (4, 4)]
Success
Path taken by agent for package location: (2, 4) - 
[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (4, 4)]
Success
Path taken by

In [36]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "browser"

In [37]:
grid = np.zeros((n, n))

In [40]:
# Visualization function
def visualize_movement(agent_positions, package_location, target_location):
    frames = []
    for i, pos in enumerate(agent_positions):

        # Is the agent carrying the package?
        carrying_package = (pos == package_location)

        # Create the grid in the animation
        grid_trace = go.Scatter(
            x=[0, n, n, 0, 0],
            y=[0, 0, n, n, 0],
            mode='lines',
            line=dict(color='black'),
            showlegend=False
        )
        
        # Create the agent (if not carrying) in the animation
        agent_trace = go.Scatter(
            x=[pos[1] + 0.5],
            y=[pos[0] + 0.5],
            mode='markers',
            marker=dict(size=15, color='red'),
            name='Agent'
        )

        # Create the package (if not picked up) in the animation
        if not carrying_package:
            package_trace = go.Scatter(
                x=[package_location[1] + 0.5],
                y=[package_location[0] + 0.5],
                mode='markers',
                marker=dict(size=12, color='blue', symbol='square'),
                name='Package'
            )
        else:
            package_trace = go.Scatter(x=[], y=[], mode='markers', name='Package (Picked Up)')

        # Create the target
        target_trace = go.Scatter(
            x=[target_location[1] + 0.5],
            y=[target_location[0] + 0.5],
            mode='markers',
            marker=dict(size=12, color='green', symbol='star'),
            name='Target'
        )
        
        frame = go.Frame(data=[grid_trace, agent_trace, package_trace, target_trace])
        frames.append(frame)

    return frames

In [45]:
# Function to simulate agent's movement 
# Simulate agent movement
def simulate_agent_movement():
    agent_row, agent_col, package_row, package_col = get_starting_locations()
    carrying = 0
    path = [(agent_row, agent_col)]
    for step in range(max_steps_per_episode):
        action_index = get_next_action(agent_row, agent_col, package_row, package_col, carrying, 1.0)
        agent_row, agent_col = get_next_location(agent_row, agent_col, action_index)
        path.append((agent_row, agent_col))
        if (agent_row, agent_col) == (package_row, package_col) and not carrying:
            carrying = 1
        if is_terminal_state(agent_row, agent_col, carrying):
            break

    
    
    return path, (package_row, package_col)

In [185]:
# Simulate agent movement
agent_positions, package_location = simulate_agent_movement()

# Visualize the agent's movement
frames = visualize_movement(agent_positions, package_location, (n - 1, n - 1))

In [186]:
fig = go.Figure(
    data=[frames[0].data[0], frames[0].data[1], frames[0].data[2], frames[0].data[3]],
    layout=go.Layout(
        xaxis=dict(range=[-1, n+1], showgrid=True, zeroline=False, showticklabels=False),
        yaxis=dict(range=[-1, n+1], showgrid=True, zeroline=False, showticklabels=False),
        title="Agent Movement Simulation",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play", method="animate", args=[None, {"frame": {"duration":500, "redraw": True}, "fromcurrent": True}]),
                    dict(label="Pause", method="animate", args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}])]
        )]
    ),
    frames=frames
)

# Show the animation
fig.show()